## Introduction

This Project explores data on Movies in the Box Office for the weekends of 2022. Columns are; 

| Columns |
| ------- |
| Date    |
| Occassion; Special weekends or not |
| Gross Revenue of Top 10 Movies in the weekend |
| Change in Top 10 Gross Revenue from last weekend |
| Overall Gross Revenue So far |
| Overall Gross Revenue in the weekend |
| Change in Overall Gross Revenue from last weekend |
| Number of releases that weekend |
| Top  movie released that weekend |
| Week Number |



### Steps Taken
1. Scrape data from the Box Office Website using requests and BeautifulSoup
2. Save Data in a CSV
3. Clean Data in Python using Pandas and other libraries
4. Visualize the **cleaned** data in Power BI

## Scrape Data and Save in CSV

In [207]:
#Import necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [208]:
#website page containing the data
url = 'https://www.boxofficemojo.com/weekend/by-year/2022/'

In [209]:
#request data from the website
request = requests.get(url)
content = request.text

In [210]:
soup = BeautifulSoup(content)

In [211]:
#find all information that is in rows in a table
rows = soup.find_all('tr')

In [212]:
#gather the data in the rows in the html table into an actual table
appended_data = []
for row in rows:
    data_row = {}
    data = row.find_all('td')
    if len(data) == 0:
        continue
    if len(data[0].find_all('span')) > 0:
        #special weekend
        data_row['occasion'] = data[0].find_all('span')[0].text
        data_row['date'] = data[0].find_all('a')[0].text
    else:
        #normal weekend
        data_row['occasion'] = ""
        data_row['date'] = data[0].text
    data_row['top10_gross'] = data[1].text
    data_row['top10_gross_change'] = data[2].text
    data_row['overall_gross'] = data[3].text
    data_row['overall_gross_change'] = data[4].text
    data_row['num_releases'] = data[5].text
    data_row['top_release'] = data[6].text
    data_row['week_no'] = data[10].text
    appended_data.append(data_row)
weekend_data = pd.DataFrame(appended_data, columns = ['date','occasion', 'top10_gross', 'top10_gross_change', 'overall_gross', 'overall_gross_change', 'num_releases', 'top_release', 'week_no']) 

### Make Data Suitable by cleaning and converting to appropriate Date Type

In [339]:
#check the data
weekend_data.head()

,date,occasion,top10_gross,top10_gross_change,overall_gross,overall_gross_change,num_releases,top_release,week_no
0,"Dec 30-Jan 2, 2023",New Year's long wknd,"$134,622,744",-,"$136,047,635",-,39,Avatar: The Way of Water,52
1,"Dec 30-Jan 1, 2023",,"$102,074,088",+8.6%,"$103,224,144",+9%,41,Avatar: The Way of Water,52
2,Dec 23-25,,"$93,952,740",-37.9%,"$94,711,196",-38.1%,35,Avatar: The Way of Water,51
3,Dec 23-26,Christmas long wknd,"$141,716,017",-,"$142,814,390",-,33,Avatar: The Way of Water,51
4,Dec 16-18,World Cup (Qatar),"$151,354,815",+355.1%,"$152,947,812",+305.6%,50,Avatar: The Way of Water,50


In [340]:
#to csv
weekend_data.to_csv("weekend_data_boxoffice")

In [341]:
#load csv
df = pd.read_csv('weekend_data_boxoffice')

In [342]:
#drop first column
df = df.drop(df.columns[0], axis=1)

In [343]:
pd.set_option('display.max_rows', None)

In [344]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   date                  63 non-null     object
 1   occasion              15 non-null     object
 2   top10_gross           63 non-null     object
 3   top10_gross_change    63 non-null     object
 4   overall_gross         63 non-null     object
 5   overall_gross_change  63 non-null     object
 6   num_releases          63 non-null     int64 
 7   top_release           63 non-null     object
 8   week_no               63 non-null     int64 
dtypes: int64(2), object(7)
memory usage: 4.6+ KB


In [345]:
df.columns

Index(['date', 'occasion', 'top10_gross', 'top10_gross_change',
       'overall_gross', 'overall_gross_change', 'num_releases', 'top_release',
       'week_no'],
      dtype='object')

In [346]:
#overall gross and top 10 gross should change to float
#drop $ sign
df['top10_gross'] = df['top10_gross'].str.replace('$', ' ').str.replace(',', '').astype(float)
df["overall_gross"] = df["overall_gross"].str.replace("$", "").str.replace(",", "").astype(float)

C:\Users\Yahdii\AppData\Local\Temp\ipykernel_23540\2607029925.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['top10_gross'] = df['top10_gross'].str.replace('$', ' ').str.replace(',', '').astype(float)
C:\Users\Yahdii\AppData\Local\Temp\ipykernel_23540\2607029925.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["overall_gross"] = df["overall_gross"].str.replace("$", "").str.replace(",", "").astype(float)


In [354]:
#remove leading and trailing spaces
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [348]:
#change format of number columns
df['overall_gross_change'] = df['overall_gross_change'].str.replace('%', '')
df['top10_gross_change'] = df['top10_gross_change'].str.replace('%','')


In [349]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  63 non-null     object 
 1   occasion              15 non-null     object 
 2   top10_gross           63 non-null     float64
 3   top10_gross_change    63 non-null     object 
 4   overall_gross         63 non-null     float64
 5   overall_gross_change  63 non-null     object 
 6   num_releases          63 non-null     int64  
 7   top_release           63 non-null     object 
 8   week_no               63 non-null     int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 4.6+ KB


In [350]:
#convert date to datetime 
df['date'] = df['date'].str.split(' ').str[0]+' '+df['date'].str.split('-').str[1]

In [351]:
#specific cases dec jan 2,2023, required the dec and , to be removed
cleaned_date_str = df.loc[0, 'date'].replace('Dec', '').replace(',', '')
# Replace the original date string in the DataFrame with the cleaned version
df.loc[0, 'date'] = cleaned_date_str

cleaned_date_str = df.loc[1, 'date'].replace('Dec', '').replace(',', '')
# Replace the original date string in the DataFrame with the cleaned version
df.loc[1, 'date'] = cleaned_date_str

cleaned_date_str = df.loc[18, 'date'].replace('Sep', '').replace(',', '')
df.loc[18, 'date'] = cleaned_date_str

cleaned_date_str = df.loc[43, 'date'].replace('Apr', '').replace(',', '')
df.loc[43, 'date'] = cleaned_date_str

In [352]:
#add 2022 to the dates without year
df['date'] = df['date'].apply(lambda x: x + " 2022"  if "2023" not in x else x)

In [355]:
#convert date to datetime format
df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%b %d %Y').strftime('%d/%m/%Y'))

In [356]:
df

,date,occasion,top10_gross,top10_gross_change,overall_gross,overall_gross_change,num_releases,top_release,week_no
0,02/01/2023,New Year's long wknd,134622744.0,-,136047635.0,-,39,Avatar: The Way of Water,52
1,01/01/2023,NaN,102074088.0,+8.6,103224144.0,+9,41,Avatar: The Way of Water,52
2,25/12/2022,NaN,93952740.0,-37.9,94711196.0,-38.1,35,Avatar: The Way of Water,51
3,26/12/2022,Christmas long wknd,141716017.0,-,142814390.0,-,33,Avatar: The Way of Water,51
4,18/12/2022,World Cup (Qatar),151354815.0,+355.1,152947812.0,+305.6,50,Avatar: The Way of Water,50
5,11/12/2022,World Cup (Qatar),33258801.0,-32.4,37708268.0,-28.4,58,Black Panther: Wakanda Forever,49
6,04/12/2022,Post-ThanksgivingWorld Cup (Qatar),49166499.0,-45.2,52669526.0,-43.8,54,Black Panther: Wakanda Forever,48
7,27/11/2022,Thanksgiving 3-DayWorld Cup (Qatar),89735916.0,-9.4,93780231.0,-6.9,49,Black Panther: Wakanda Forever,47
8,27/11/2022,Thanksgiving 4-DayWorld Cup (Qatar),106837323.0,-,111394577.0,-,36,Black Panther: Wakanda Forever,47
9,27/11/2022,Thanksgiving 5-DayWorld Cup (Qatar),128658959.0,-,134030714.0,-,37,Black Panther: Wakanda Forever,47


In [357]:
#convert to date time date type
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)

C:\Users\Yahdii\AppData\Local\Temp\ipykernel_23540\1932662626.py:2: UserWarning: Parsing '25/12/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
C:\Users\Yahdii\AppData\Local\Temp\ipykernel_23540\1932662626.py:2: UserWarning: Parsing '26/12/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
C:\Users\Yahdii\AppData\Local\Temp\ipykernel_23540\1932662626.py:2: UserWarning: Parsing '18/12/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
C:\Users\Yahdii\AppData\Local\Temp\ipykernel_23540\1932662626.py:2: UserWarning: Parsing '27/11/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consis

In [358]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  63 non-null     datetime64[ns]
 1   occasion              15 non-null     object        
 2   top10_gross           63 non-null     float64       
 3   top10_gross_change    63 non-null     object        
 4   overall_gross         63 non-null     float64       
 5   overall_gross_change  63 non-null     object        
 6   num_releases          63 non-null     int64         
 7   top_release           63 non-null     object        
 8   week_no               63 non-null     int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 4.6+ KB


### Convert to CSV for PowerBI

In [359]:
df.to_csv('movie.csv')

In [313]:
pwd

'C:\\Users\\Yahdii'